In [1]:
import pandas as pd
import spacy
import os
import unicodedata
import re
from typing import Dict, List, Iterator, Union, Callable
import random
random.seed(42)
import numpy as np
from datetime import datetime

import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
torch.manual_seed(42)
torch.cuda.manual_seed(42)


from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.tokenizers import Token
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data import Instance
from allennlp.data.iterators import BucketIterator, BasicIterator
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.models import Model

from allennlp.training.trainer import Trainer

MAX_LENGTH = 10


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
home = os.path.expanduser("~")
data_dir = home + '/data/workshop_data/'
data = 'cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/formatted_movie_lines.txt'
sp_data = 'southpark_full.csv'
log_dir = home + '/data/logs/'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
if not os.path.exists(data_dir + data):
    try:
        import boto3
        s3 = boto3.resource('s3')
        s3.Bucket('workshopqcondata').download_file(data, data_dir + data)
    except:
        print('Please download the data using the fetch notebooks')

In [3]:
class DialogueReader(DatasetReader):
    """
    DatasetReader for reading dialogue data from a csv
    """

    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.vocab = None

    def text_to_instance(self, query: List[Token], reply: List[Token] = None) -> Instance:
        query = TextField(query, self.token_indexers)
        fields = {"query": query}
        if reply is not None:
            reply = TextField(reply, self.token_indexers)
            fields["reply"] = reply

        return Instance(fields)
    
    @staticmethod
    def unicode_to_ascii(s):
        return ''.join(
            c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn')
    
    # Lowercase, trim, and remove non-letter characters
    @staticmethod
    def normalize_string(s):
        s = DialogueReader.unicode_to_ascii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = s.replace(" . . .", " ...")
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
        s = re.sub(r"\s+", r" ", s).strip()
        return s

    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path, header=None, names=['query', 'reply'], sep='\t')
        for _, row in df.iterrows():
            query = self.normalize_string(row['query']).split(' ')
            reply = self.normalize_string(row['reply']).split(' ')
            if len(query) >= MAX_LENGTH or len(reply) >= MAX_LENGTH:
                continue
            if self.vocab is not None:
                if np.any([self.vocab.get_token_index(x) == 1 for x in query]):
                    # print(f"skipped {query} {[self.vocab.get_token_index(x) for x in query]}")
                    continue
            query = [Token(word) for word in query] + [Token('#EOS#')]
            # print(self.vocab)
            if self.vocab is not None:
                if np.any([self.vocab.get_token_index(x) == 1 for x in reply]):
                    # print(f"skipped {reply} {[self.vocab.get_token_index(x) for x in reply]}")
                    continue
            reply = [Token(word) for word in reply] + [Token('#EOS#')]
            yield self.text_to_instance(query, reply)
 

class CartmanReader(DialogueReader):

    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path, sep='\t')
        for i in range(len(df)):
            if (df['character'][i] != 'Cartman' or
                df['episode'][i] != df['episode'][i] or
                df['setting'][i] != df['setting'][i] or
                not pd.notnull(df['spoken'][i-1]) or
                not pd.notnull(df['spoken'][i])):
                    continue
            query = self.normalize_string(df['spoken'][i-1]).split(' ')
            reply = self.normalize_string(df['spoken'][i]).split(' ')
            if reply[0] == 'what': continue
            if len(reply) < 3:
                continue
            if len(query) >= MAX_LENGTH or len(reply) >= MAX_LENGTH:
                continue
            if self.vocab is not None:
                if np.any([self.vocab.get_token_index(x) == 1 for x in query]):
                    # print(f"skipped {query} {[self.vocab.get_token_index(x) for x in query]}")
                    continue
            query = [Token(word) for word in query] + [Token('#EOS#')]
            # print(self.vocab)
            if self.vocab is not None:
                if np.any([self.vocab.get_token_index(x) == 1 for x in reply]):
                    # print(f"skipped {reply} {[self.vocab.get_token_index(x) for x in reply]}")
                    continue
            reply = [Token(word) for word in reply] + [Token('#EOS#')]
            yield self.text_to_instance(query, reply)

In [4]:
reader1 = DialogueReader()
dialogue_dataset1 = reader1.read(data_dir + data)
reader2 = CartmanReader()
cartman_dataset1 = reader2.read(data_dir + sp_data)

sos_token = '#SOS#'
vocab = Vocabulary.from_instances(dialogue_dataset1 + cartman_dataset1, min_count={'tokens': 2})
vocab.add_token_to_namespace(sos_token)

67210it [00:36, 1851.87it/s]
1496it [00:02, 738.90it/s]
100%|██████████| 68706/68706 [00:00<00:00, 167484.12it/s]


12092

In [5]:
reader = DialogueReader()
reader.vocab = vocab
dialogue_dataset = reader.read(data_dir + data)

61335it [00:38, 1592.44it/s]


In [6]:
vocab.print_statistics()
print(vocab.get_vocab_size())



----Vocabulary Statistics----


Top 10 most frequent tokens in namespace 'tokens':
	Token: #EOS#		Frequency: 137412
	Token: .		Frequency: 85275
	Token: ?		Frequency: 46420
	Token: you		Frequency: 31329
	Token: i		Frequency: 23918
	Token: s		Frequency: 13472
	Token: what		Frequency: 13354
	Token: !		Frequency: 13187
	Token: it		Frequency: 12431
	Token: the		Frequency: 11676

Top 10 longest tokens in namespace 'tokens':
	Token: aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa		length: 33	Frequency: 1
	Token: phreakphreakphreakphreakphreak		length: 30	Frequency: 1
	Token: mmmmmmmmmmmmmnnnnnnmmmmmmmm		length: 27	Frequency: 1
	Token: bebebudjuhbluhbluhbluhbluh		length: 26	Frequency: 1
	Token: bwolololololololololololo		length: 25	Frequency: 1
	Token: dudedudedudedudedudedude		length: 24	Frequency: 1
	Token: aaaaahhnnnhaaaaaannnhhh		length: 23	Frequency: 1
	Token: aaaahhhhnnnahhhnnnggnnn		length: 23	Frequency: 1
	Token: aaaaannnnnuuunnnuhhhhh		length: 22	Frequency: 1
	Token: rowrrowrrowrrowrrowr		length:

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding: TextFieldEmbedder, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = PytorchSeq2SeqWrapper(nn.GRU(
            hidden_size, hidden_size, n_layers,
            dropout=(0 if n_layers == 1 else dropout), bidirectional=True, batch_first=True))

    def forward(self, query: Dict[str, torch.Tensor]):
        # Convert word indexes to embeddings
        mask = get_text_field_mask(query)
        
        embeddings = self.embedding(query)
        
        encoder_out = self.gru(embeddings, mask)
        
        # Sum bidirectional GRU outputs
        outputs = encoder_out[:, :, :self.hidden_size] + encoder_out[:, : ,self.hidden_size:]
        
        # Return output and final hidden state
        return outputs
    
# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(-1, encoder_output.size(1), -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)
        # Transpose max_length and batch_size dimensions
        # attn_energies = attn_energies.t()
        # print(attn_energies.shape)

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(2)
    
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding: TextFieldEmbedder, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(
            hidden_size, hidden_size, n_layers, 
            dropout=(0 if n_layers == 1 else dropout), batch_first=True)
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, encoder_outputs, last_hidden=None):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding({'tokens': input_step})
        embedded = self.embedding_dropout(embedded)
        
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)#.squeeze(2)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = torch.sum(attn_weights * encoder_outputs, dim=1)
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(1)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        # Return output and final hidden state
        return output, hidden

In [8]:
class Chatbot(Model):
    def __init__(self, encoder: EncoderRNN, decoder: LuongAttnDecoderRNN,
                 vocab: Vocabulary, sos_token: Token, batch_size: int,
                 max_target_len: int, teacher_forcing_ratio):
        super().__init__(vocab)
        self.encoder = encoder
        self.decoder = decoder
        self.vocab = vocab
        self.sos_token = sos_token
        self.max_target_len = max_target_len
        self.teacher_forcing_ratio = teacher_forcing_ratio
        
    @property
    def device(self):
        return next(self.parameters()).device

    def forward(self, query: Dict[str, torch.Tensor], reply: Dict[str, torch.Tensor] = None):
        
        max_target_len = self.max_target_len if reply is None else reply['tokens'].shape[1]
        output = {}
        encoder_outputs = encoder(query)
        mask = get_text_field_mask(query)
        selector = mask.sum(dim=1) - 1
        batch_size = query['tokens'].shape[0]
        #decoder_hidden = encoder_outputs[:, selector]
        decoder_hidden = torch.stack([encoder_outputs[idx, index, :] for idx, index in enumerate(selector)], dim=0)
        #print(decoder_hidden.shape)
        decoder_hidden = torch.stack((decoder_hidden, decoder_hidden), dim=0)
        #print(decoder_hidden.shape, encoder_outputs.shape, selector.shape)
        #decoder_hidden = None #final_states[:decoder.n_layers] # [0,2]] + final_states[[1,3]]
        # for i in range(batch_size):
        #     decoder_hidden.append(encoder_outputs[i, selector[i]])
        # print(decoder_hidden[0].shape)
        # decoder_hidden = torch.stack(decoder_hidden, dim=0)
        decoder_input = torch.LongTensor([[self.vocab.get_token_index(self.sos_token)] for _ in range(batch_size)])
        decoder_input = decoder_input.to(self.device)
        # Determine if we are using teacher forcing this iteration
        use_teacher_forcing = True if random.random() < self.teacher_forcing_ratio else False
        ret = []
        # Forward batch of sequences through decoder one time step at a time
        if use_teacher_forcing and reply is not None:
            loss = 0
            for t in range(max_target_len):
                decoder_output, decoder_hidden = self.decoder(
                    decoder_input, encoder_outputs, decoder_hidden)
                # Teacher forcing: next input is current target
                decoder_input = reply['tokens'][:, t].unsqueeze(1)
                ret.append(decoder_output)# torch.LongTensor([topi[i][0] for i in range(batch_size)]))
                # Calculate and accumulate loss
        else:
            for t in range(max_target_len):
                decoder_output, decoder_hidden = self.decoder(
                    decoder_input, encoder_outputs, decoder_hidden)
                # print(decoder_hidden.shape, encoder_outputs.shape, decoder_input.shape)
                # No teacher forcing: next input is decoder's own current output
                # _, topi = decoder_output.topk(1)
                decoder_input = decoder_output.argmax(dim=1).unsqueeze(1)
                # print(decoder_output.shape, decoder_input.shape)
                # print(decoder_input)
                # decoder_input = torch.LongTensor([[topi[i][0]] for i in range(batch_size)])
                # print(decoder_output.shape, decoder_input.shape)
                # print(decoder_input)
                ret.append(decoder_output)
                decoder_input = decoder_input.to(self.device)
        ret = torch.stack(ret, dim=1)
        # print(ret)
        # print(reply['tokens'])
        if reply is not None:
            # print(ret.shape, reply['tokens'].shape)
            mask = get_text_field_mask(reply)
            # mask = mask.float()*((reply['tokens'] == self.vocab.get_token_index('#EOS#') |
            #                     (reply['tokens'] == self.vocab.get_token_index('.')).float() * -.75 + 1)
            loss = sequence_cross_entropy_with_logits(ret, reply['tokens'], mask)
            output["loss"] = loss
            # print(f'Averaged loss {loss}')
            #mask = mask.float()        
            #loss = sequence_cross_entropy_with_logits(ret, reply['tokens'], mask, average=None)
            #output["loss"] = (loss*mask.sum(1) / mask.sum(1).mean()).mean()
            # print(f'Calculated loss {output["loss"]}')
        output["reply"] = ret.argmax(dim=2)
        return output
        

In [9]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.2
batch_size = 64
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_epochs = 6
print_every = 1
save_every = 500
device = 'cuda:0'

print('Building encoder and decoder ...')
# Initialize word embeddings


token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=hidden_size)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size,word_embeddings, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, word_embeddings, hidden_size,
                              vocab.get_vocab_size('tokens'), decoder_n_layers, dropout)
# Use appropriate device
print('Models built and ready to go!')
model = Chatbot(encoder, decoder, vocab, sos_token, batch_size, MAX_LENGTH, teacher_forcing_ratio)
model.to(device)
optimizer = optim.Adam([
                {'params': model.encoder.parameters()},
                {'params': [param for name, param in model.decoder.named_parameters() 
                            if 'embedding' not in name], 'lr': 5*learning_rate}
                        ], lr=learning_rate)

iterator = BasicIterator(batch_size=batch_size)#, sorting_keys=[("query", "num_tokens")])

iterator.index_with(vocab)
log_path = log_dir + datetime.now().ctime()
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=dialogue_dataset,
                  grad_clipping=clip,
                  patience=1,
                  num_epochs=n_epochs,
                  cuda_device=int(device[-1]),
                  serialization_dir = log_path,
                  histogram_interval=1000)

Building encoder and decoder ...
Models built and ready to go!


In [10]:
trainer.train()

loss: 4.3637 ||: 100%|██████████| 959/959 [02:17<00:00,  7.39it/s]
loss: 3.7336 ||: 100%|██████████| 959/959 [02:15<00:00,  7.07it/s]
loss: 3.5115 ||: 100%|██████████| 959/959 [02:14<00:00,  7.12it/s]
loss: 3.3494 ||: 100%|██████████| 959/959 [02:15<00:00,  7.09it/s]
loss: 3.2107 ||: 100%|██████████| 959/959 [02:16<00:00,  7.02it/s]
loss: 3.0841 ||: 100%|██████████| 959/959 [02:22<00:00,  7.80it/s]


{'best_epoch': 5,
 'peak_cpu_memory_MB': 3262.352,
 'peak_gpu_0_memory_MB': 2544,
 'training_duration': '00:13:44',
 'training_start_epoch': 0,
 'training_epochs': 5,
 'epoch': 5,
 'training_loss': 3.084123701447615,
 'training_cpu_memory_MB': 3262.352,
 'training_gpu_0_memory_MB': 2544}

In [11]:
def get_reply(predictor, question):
    vec = predictor.predict(question)['reply']
    vec = [str(vocab.get_token_from_index(x)) for x in vec]
    ret = ''
    for word in vec:
        if word == '#EOS#':
            break
        ret += word + ' '
    ret = ret[:-1]
    return ret

In [12]:
#model.eval()
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
print(get_reply(predictor, "what is my name?"))
print(get_reply(predictor, "what is your name?"))
print(get_reply(predictor, "where am i?"))
print(get_reply(predictor, "who are you?"))
print(get_reply(predictor, "where are you from?"))
print(get_reply(predictor, "why are you going to do that?"))
print(get_reply(predictor, "i hate you!"))
print(get_reply(predictor, "are you mad?"))

victor .
victor .
i m going to be a taxi .
i m fine .
i m going to get out .
i m going to be fine .
what ?
i m fine .


In [13]:
# Here's how to save the model.
with open("/tmp/dialogue_bot.th", 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files("/tmp/vocabulary")

In [14]:
with open("/tmp/dialogue_bot.th", 'rb') as f:
    model.load_state_dict(torch.load(f)

SyntaxError: unexpected EOF while parsing (<ipython-input-14-8d35d201e137>, line 2)

In [ ]:
reader = CartmanReader()
reader.vocab = vocab
cartman_dataset = reader.read(data_dir + sp_data)

In [ ]:
print(cartman_dataset[1027])

In [ ]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=cartman_dataset,
                  grad_clipping=clip,
                  patience=1,
                  num_epochs=14,
                  cuda_device=int(device[-1]),
                  serialization_dir=log_path,
                  histogram_interval=1000)

In [ ]:
model.train()
trainer.train()

In [ ]:
# Here's how to save the model.
with open("/tmp/cartman_bot.th", 'wb') as f:
    torch.save(model.state_dict(), f)

In [ ]:
model.eval()
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
print(get_reply(predictor, "what up?"))
print(get_reply(predictor, "how are you?"))
print(get_reply(predictor, "is this real?"))
print(get_reply(predictor, "how is kenny?"))
print(get_reply(predictor, "how is kyle?"))
print(get_reply(predictor, "i hate you!"))
print(get_reply(predictor, "cartman is stupid!"))
print(get_reply(predictor, "shut up!"))